<a href="https://colab.research.google.com/github/Changyoon-Lee/mulcamB_1/blob/master/%EC%9D%98%EC%82%AC%EB%A1%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_ver_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 의사록에서 PDF파일 불러오기


In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
total_list = [] # 전체 리스트를 가져오고]

for i in range(32,33):
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    query = {
        'pageIndex' : i
      }
    resp = requests.get(url, query)
    soup = BeautifulSoup(resp.content, 'lxml')
    tag = soup.select('div.fileGoupBox a')
    # print(len(tag))
    total_list.extend(tag)

    pdf_list = []
    for idx,t in enumerate(total_list):
      if total_list[idx].attrs['title'][-3:] == 'pdf': #tag에서 .attrs['href'] href만 가져오기
        pdf_list.append(total_list[idx]) #.attrs['href']

In [ ]:
pdf_href_list = []
for i in pdf_list:
  pdf_href_list.append(i.attrs['href'])

# pdf_title_list = []
# for i in pdf_list:
#   pdf_title_list.append(i.attrs['title'])

site = 'https://www.bok.or.kr'

url = []
for i in range(len(pdf_href_list)):
   url.append(site + pdf_href_list[i])

In [ ]:
for i in range(len(url)):
  mpb = requests.get(url[i])
  with open('/content/gdrive/My Drive/MPB/{}.pdf'.format(i), 'wb') as f: #저장하는 폴더
    f.write(mpb.content)

## pdf file ->text file 저장


In [ ]:
pip install tika

In [ ]:
import re
from tika import parser #https://github.com/chrismattmann/tika-python
import shutil
from os import listdir
from os.path import isfile, join
import os

In [ ]:
def pdf2txt (source_folder, output_folder) :#pdf파일을 text파일로 변경해 주는 함수
    # 지정 폴더 내 파일 목록 조회 (파일만)
    # in ~~~ 는 폴더 내 다른 서브퓰더가 있는 경우를 처리하기 위함.
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for pdf in pdf_files :
        pdf_filepath = source_folder + pdf
        
        try:    
            pdf_filepath = source_folder + pdf
            # pdf 파일을 text로 변환
            print(pdf_filepath)
            parsedPDF = parser.from_file(pdf_filepath)["content"]

        except Exception as e:
            parasedPDF = ' '
            print(f'{pdf} 에러 : {e}')
        
        ouput_filepath = (output_folder + pdf).replace('.pdf', '.txt')
        with open(ouput_filepath, 'w', encoding='utf-8') as f :
            print(ouput_filepath)
            f.write(parsedPDF)
            f.close()

In [ ]:
# pdf->txt파일로 변환
base = os.getcwd() # 현재 작업 폴더 얻기
source = os.path.join(base, '/content/gdrive/My Drive/') #PDF파일이 있는 드라이브로 설정
output = os.path.join(base, '/content/gdrive/My Drive/MPB_Text/')#Text파일이 있는 드라이브로 저장
pdf2txt(source, output) #pdf->text 파일로 변경

## txt데이터 가져오기

In [ ]:
k = []
for i in range(7,272):
    filename = r'C:\Users\junshey\jupyter\MPB_Text\{}.txt'.format(i) # 저장 폴더
    k.append(filename)
len(k)


In [ ]:
k[1]

In [ ]:
t = []
for i in range(len(k)):
    with open(k[i],'r',encoding='UTF-8') as file_object:
        contents = file_object.read()
        t.append(contents)

In [ ]:
df_text = pd.DataFrame(t, columns=['text'])

## date 

In [ ]:
import requests
from bs4 import BeautifulSoup

date_list = [] # 전체 리스트를 가져오고] 
for i in range(5,33):
    url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761'
    query = {
        'pageIndex' : i
      }
    resp = requests.get(url, query).text
    soup = BeautifulSoup(resp, 'html.parser') #resp.conetent 로 뽑으면 Tag형태로 나오기 때문에 content를 쓰지 않고 리스트 형태로 전환 (날짜만 뽑기 위해)
    tag = soup.select('div span.titlesub')
    date_list.extend(tag)


In [ ]:
date_1 = [] # join함수를 써서 list->str로 변환
for t in date_list:
  date_1.append('\n'.join(t))
len(date_1)

date_2 = date_1[7:-8] # 기존의 날짜로 정리

date_3 = []
for d in range(len(date_2)):
  date_3.append(date_2[d].split('('))

date_4 = []
for t in date_3:
  date_4.append(t[2][:-1])

date_5 = []
for d in date_4:
  if d[-1] == '.':
    date_5.append(d[:-1])
  else:
    date_5.append(d)

len(date_5)

In [ ]:
df_date = pd.DataFrame(date_5, columns=['date'])

## 두개파일 concat

In [ ]:
df = pd.concat([df_date,df_text], axis = 1)